In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('raport_awarii.csv', encoding='utf-8-sig', sep=";")

df

,Zawiadomienie,Rodzaj zawiadomienia,Utworzono dnia,Początek zakłócenia,Pocz. zakłóc. (godz.),Koniec zakłócenia,Koniec zakłóc.(godz.),Przestój,Urządzenie,Lokalizacja funkc.,...,Nr zlecenia,Wpływ,Problem,Kod uszkodzenia,Tekst dot. szkody,Kod przyczyny,Ozn. lokaliz. funkc.,Line functional location,Ozn. lokaliz. funkc..1,Oznaczenie obiektu
0,12476707,PT,02.01.2025,02.01.2025,06:30:17,NaN,00:00:00,NaN,10117891.0,PLPA-PR-U11-010703-003,...,5.000021e+10,NaN,NaN,NaN,NaN,NaN,MODELATOR RAFFAELLO,PLPA-PR-U11-010703,RAFFAELLO LINIA 2,1 - Wałek czyszczący formy
1,12476648,1P,02.01.2025,02.01.2025,10:27:15,02.01.2025,12:40:14,X,10226078.0,PLPA-PR-U12-010801-002,...,5.000021e+10,2.0,ZPM5,105.0,Zatkanie,108.0,KINDER CARDS MODELATOR,PLPA-PR-U12-010801,KINDER CARDS,1-Instalacja tech. kremu ciemnego
2,12476708,PT,02.01.2025,02.01.2025,06:32:26,NaN,00:00:00,NaN,10117907.0,PLPA-PR-U11-010703-003,...,5.000021e+10,NaN,NaN,NaN,NaN,NaN,MODELATOR RAFFAELLO,PLPA-PR-U11-010703,RAFFAELLO LINIA 2,1-Składacz form 1 002104
3,12476709,PT,02.01.2025,02.01.2025,06:33:21,NaN,00:00:00,NaN,10117921.0,PLPA-PR-U11-010703-003,...,5.000021e+10,NaN,NaN,NaN,NaN,NaN,MODELATOR RAFFAELLO,PLPA-PR-U11-010703,RAFFAELLO LINIA 2,1-Składacz form 2 005066
4,12476881,1P,02.01.2025,02.01.2025,08:21:02,NaN,00:00:00,X,10116830.0,PLPA-PR-U11-010717,...,NaN,1.0,NaN,NaN,NaN,NaN,LINIA PAKUJĄCA ERMETICO RAFFAELLO,PLPA-PR-U11,UGP11 RAFFAELLO,1-Maszyna pakująca Ermetico 2 003231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6304,12707605,PT,30.06.2025,30.06.2025,13:58:44,NaN,00:00:00,NaN,10226868.0,PLPA-PR-U12-010801-003,...,5.000023e+10,NaN,NaN,NaN,NaN,NaN,KINDER CARDS PAKOWANIE,PLPA-PR-U12-010801,KINDER CARDS,2-Pakowaczka pozioma
6305,12707606,PT,30.06.2025,30.06.2025,13:59:42,NaN,00:00:00,NaN,10226877.0,PLPA-PR-U12-010801-003,...,5.000023e+10,NaN,NaN,NaN,NaN,NaN,KINDER CARDS PAKOWANIE,PLPA-PR-U12-010801,KINDER CARDS,2-Pakowaczka pozioma
6306,12707607,PT,30.06.2025,30.06.2025,14:03:23,NaN,00:00:00,NaN,10226889.0,PLPA-PR-U12-010801-003,...,5.000023e+10,NaN,NaN,NaN,NaN,NaN,KINDER CARDS PAKOWANIE,PLPA-PR-U12-010801,KINDER CARDS,2-Pakowaczka pozioma
6307,12707608,PT,30.06.2025,30.06.2025,14:04:44,NaN,00:00:00,NaN,10226883.0,PLPA-PR-U12-010801-003,...,5.000023e+10,NaN,NaN,NaN,NaN,NaN,KINDER CARDS PAKOWANIE,PLPA-PR-U12-010801,KINDER CARDS,2-Pakowaczka pozioma


In [4]:
df1 = pd.read_csv('test.csv', encoding='utf-8-sig', sep=";")

df1

,miesiąc,tydzień,Linia,Rodzina,QL/TOT,QL/TOT2,% Dvt,Zmiana,Zmiana3,Q/Zm.,...,% E/LINIA,% E/LINIA14,% E/Prac.,% E/Prac.15,% Zysku,% Zysku16,suma,% scrap,% scrap17,oszczędności
0,NaN,NaN,NaN,NaN,Akt,Pln,Produk.,Czysty,Prg,Standard,...,Eff,Obb,Eff,Obb,Eff,Obb,oszczęd,Eff,Std.,Godz./Prac.
1,02.09 - 08.09,36.0,10401.0,I210,"1 398,797","1 397,894","0,065","9,991","9,985","158,976",...,"88,065","88,060","74,111","88,060","88,065","88,064","-429,751","3,082","3,102","429,751"
2,NaN,NaN,10401.0,I211,"108,192","108,115","0,071","0,773","0,772","158,976",...,"88,070","88,060","96,449","88,060","88,070","88,064","-410,412","2,819","2,900","-19,340"
3,NaN,NaN,10401.0,I212,"1 079,160","1 078,577","0,054","7,707","7,704","158,976",...,"88,077","88,060","134,293","88,060","88,077","88,064","684,778","2,657","3,202","-1 095,190"
4,09.09 - 15.09,37.0,10401.0,I211,"477,147","476,201","0,199","3,408","3,401","158,976",...,"88,065","88,060","79,148","88,060","88,065","88,064","570,817","2,898","2,900","113,961"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,14.04 - 20.04,16.0,10401.0,I214,"791,804","790,940","0,109","5,596","5,650","158,976",...,"89,000","88,060","88,314","88,060","89,000","88,064","2 737,211","3,098","3,100","-4,899"
91,NaN,NaN,10401.0,I215,"796,707","795,811","0,113","5,731","5,684","158,975",...,"87,451","88,060","88,069","88,060","87,451","88,064",NaN,NaN,NaN,NaN
92,21.04 - 27.04,17.0,10401.0,I210,"861,889","848,387","1,591","6,831","6,060","158,976",...,"79,363","88,060","82,497","88,060","79,363","88,064","2 615,151","3,100","3,100","122,060"
93,NaN,NaN,10401.0,I211,"110,501","106,012","4,235","0,789","0,757","158,976",...,"88,124","88,060","47,782","88,060","88,124","88,064",NaN,NaN,NaN,NaN
